In [7]:
def read(f):
    for l in open(f):
        yield eval(l)

from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model

comments = []
usernames = []
count = 0
for i in read('opt_dataset/small_opt.json'):
    comments.append(i['comment'])
    usernames.append(i['userName'])
    
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stemmer = PorterStemmer()

for i in comments:
    r = ''.join([c for c in i.lower() if not c in punctuation])
    for w in r.split():
        # w = stemmer.stem(w)
        wordCount[w] += 1

labels = []
from sets import Set
vocab = Set()
for i in range(600):
    r = ''.join([c for c in comments[i].lower() if not c in punctuation])
    for w in r.split():
        vocab.add(w)
numberpos = 0.0
numberneg = 0.0
for l in open('opt_dataset/labels_1-700.txt'):
    if(eval(l) == 1):
        labels.append(1)
        numberpos += 1
    else:
        labels.append(0)
        numberneg += 1
posprob = numberpos/(numberpos + numberneg)
negprob = 1-posprob

In [8]:
# Multinomial naive bayes!

pos_wordCount_mult = defaultdict(int)
neg_wordCount_mult = defaultdict(int)
for w in vocab:
    pos_wordCount_mult[w] = 0
    neg_wordCount_mult[w] = 0
numberpos_mult = 0
numberneg_mult = 0
# calculating the probabilites
for i in range(600):
    r = ''.join([c for c in comments[i].lower() if not c in punctuation])
    bag = r.split()
    for w in bag:
        if(labels[i] == 1):
            pos_wordCount_mult[w] += 1
            numberpos_mult += 1
        else:
                neg_wordCount_mult[w] += 1
                numberneg_mult += 1
                
pos_wordprob_mult = defaultdict(float)
neg_wordprob_mult = defaultdict(float)

for w in vocab:
    pos_wordprob_mult[w] = 1.0*(pos_wordCount_mult[w] + 1)/(len(vocab) + numberpos_mult)
    neg_wordprob_mult[w] = 1.0*(neg_wordCount_mult[w] + 1)/(len(vocab) + numberneg_mult)

In [9]:
import math
def predpos_mult(words):
    sum = 0.0
    for w in words:
        if(w in vocab):
            sum += math.log(pos_wordprob_mult[w])
    return (math.log(posprob) + sum)
            
def predneg_mult(words):
    sum = 0.0
    for w in words:
        if(w in vocab):
            sum += math.log(neg_wordprob_mult[w])
    return (math.log(negprob) + sum)
            
predictions_mult = []
LLP = []
LLN = []
for i in range(len(comments)):
    r = ''.join([c for c in comments[i].lower() if not c in punctuation])
    words = r.split()
    x = predpos_mult(words)
    y = predneg_mult(words)
    LLP.append(x)
    LLN.append(y)
    if(x > y):
        predictions_mult.append(1)
    else:
        predictions_mult.append(0)

In [10]:
# Semi supervised learning, Some kind of expectation maximization...
num_iter = 0
while(num_iter < 4):
    pos_wordCount_mult = defaultdict(int)
    neg_wordCount_mult = defaultdict(int)
    for w in vocab:
        pos_wordCount_mult[w] = 0
        neg_wordCount_mult[w] = 0
    numberpos_mult = 0
    numberneg_mult = 0
    # calculating the probabilites
    for i in range(len(comments)):
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        bag = r.split()
        for w in bag:
            if(predictions_mult[i] == 1):
                pos_wordCount_mult[w] += 1
                numberpos_mult += 1
            else:
                    neg_wordCount_mult[w] += 1
                    numberneg_mult += 1

    pos_wordprob_mult = defaultdict(float)
    neg_wordprob_mult = defaultdict(float)

    for w in vocab:
        pos_wordprob_mult[w] = 1.0*(pos_wordCount_mult[w] + 1)/(len(vocab) + numberpos_mult)
        neg_wordprob_mult[w] = 1.0*(neg_wordCount_mult[w] + 1)/(len(vocab) + numberneg_mult)

    predictions_mult = []
    LLP = []
    LLN = []
    for i in range(len(comments)):
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        words = r.split()
        x = predpos_mult(words)
        y = predneg_mult(words)
        LLP.append(x)
        LLN.append(y)
        if(x > y):
            predictions_mult.append(1)
        else:
            predictions_mult.append(0)
    
    num_iter += 1
    print "iteration number:",num_iter

iteration number: 1
iteration number: 2
iteration number: 3
iteration number: 4


In [11]:
train_accuracy = 0.0
test_accuracy = 0.0
for i in range(700):
    if(predictions_mult[i] == labels[i]):
        if i<600:
            train_accuracy += 1
        else:
            test_accuracy += 1
    elif(i>=600  and i<700):
        print comments[i], "True label:",labels[i],"Predicted label:",predictions_mult[i]
print "Test accuracy:",test_accuracy/100,"Train accuracy:", train_accuracy/600 

I see absolutely no reason for expanding the opportunities F-1 nonimmigrant students who merely make it much harder for United States young people who have such a difficult time finding work. Please do not allow additional visas or additional time for F-1 individuals to work in the U.S. True label: 0 Predicted label: 1
As a US citizen, I personally think OPT program abuse is too prevailing to be acceptable for US public. However, I do believe there should be common ground as US should retain talented foreign students while preventing abuse Therefore I suggest the new OPT rule should be improved by these steps:  1) very clear and high enough prevailing wage requirement for employers to recruit students on OPT. 2) set up a black list of companies that abuse OPT and H1b, those in black list will be forever barred from using OPT or H1b program. 3)OPT program should only be available for students from top 50-100 US universities, the current provision "accredited Universities" is too vague. 